In [6]:
###EFP CODE###

import parmed as pmd
import numpy as np
import os, math, sys, itertools
from tabulate import tabulate

#Rotation Matrices
def rotate_to_global(molecule):
    xA, xB, xC, xD = molecule[0], molecule[1], molecule[2], molecule[3]
    vCA = (xA - xC)
    vCA /= np.linalg.norm(vCA)
    vDB = (xB - xD)
    vDB /= np.linalg.norm(vDB)
    vnorm = np.cross(vCA, vDB)
    vnorm /= np.linalg.norm(vnorm)

    z_axis_local = -vnorm
    y_axis_local = vCA
    x_axis_local = np.cross(vCA, -vnorm)
    x_axis_local /= np.linalg.norm(x_axis_local)

    r12_x = xx = x_axis_local[0]
    r12_y = yx = x_axis_local[1]
    r12_z = zx = x_axis_local[2]
        
    r13_x = xy = y_axis_local[0]
    r13_y = yy = y_axis_local[1]
    r13_z = zy = y_axis_local[2]

    cross_x = xz = z_axis_local[0]
    cross_y = yz = z_axis_local[1]
    cross_z = zz = z_axis_local[2]

    rotmat_local_to_global = np.array([[xx, yx, zx],[xy,yy,zy],[xz,yz,zz]])
    return rotmat_local_to_global

#Matrix to Euler 
def rotation_matrix_to_euler_angles(rotation_matrix):
    if rotation_matrix[2,0]!=1 or rotation_matrix[2,0]!=-1:
        y1 = -math.asin(rotation_matrix[2,0])
        y2 = math.pi - y1
        if y1 >= 0 or y2 >= 0:
            y = y1
            x = math.atan2((rotation_matrix[2, 1] / math.cos(y1)), (rotation_matrix[2, 2] / math.cos(y1)))
            z = math.atan2((rotation_matrix[1, 0] / math.cos(y1)), (rotation_matrix[0, 0] / math.cos(y1)))
        else:
            y = y2
            x = math.atan2((rotation_matrix[2, 1]/math.cos(y2)), (rotation_matrix[2, 2]/math.cos(y2)))
            z = math.atan2((rotation_matrix[1, 0]/math.cos(y2)), ((rotation_matrix[0, 0])/math.cos(y2)))
    else:
        z = 0
        if (rotation_matrix[2,0]== -1):
            y = math.pi/2
            x = z + math.atan2(rotation_matrix[0,1], rotation_matrix[0,2])
        else:
            y = -math.pi/2
            x = -z + math.atan2(-rotation_matrix[0,1], -rotation_matrix[0,2])
    return z, y, x


directory_path = './tmp'
os.makedirs(directory_path, exist_ok=True)

monomer=sys.argv[0]
#load pdb structure file 
struc = pmd.load_file('shift_z_0.0_shift_y_0.0_shift_x_0.0_psi_-10_theta_0_phi_0.pdb')

#identify key chlorophyll atoms
unique_atoms = {}
for atom in struc.atoms:
    atom_type = (atom.name, atom.residue.name, atom.residue.idx)
    if atom.name in ['NA','NB','NC','ND'] and atom_type not in unique_atoms:
        unique_atoms[atom_type] = [atom.xx, atom.xy, atom.xz]

atom_matrix = np.empty((len(unique_atoms), 6), dtype=object)

# Populate the atom matrix with atom information
for i, (atom_key, coordinates) in enumerate(unique_atoms.items()):
    atom_matrix[i, 0] = atom_key[0]  # Atom name
    atom_matrix[i, 1] = atom_key[1]  # Residue Name
    atom_matrix[i, 2] = atom_key[2]  # ResID
    atom_matrix[i, 3:] = coordinates  # X, Y, Z
    
new_atom_matrix = np.delete(atom_matrix, [0, 1, 2], axis=1)
totallines=new_atom_matrix.shape[0]
lines = totallines//4
#write new files for each chlorophyll present in the protein being studied
for j in range(lines):
    start=j*4
    end=(j+1)*4
    chl_matrix=new_atom_matrix[start:end,:]
    
    np.savetxt(f'./tmp/CHL_{j+1}.txt', chl_matrix, fmt='%s')
chl_files = [f for f in os.listdir(directory_path) if f.endswith('.txt') and os.path.isfile(os.path.join(directory_path, f))]

for chl_combination in itertools.combinations(chl_files, 2):
    chl1 = chl_combination[0]
    chl2 = chl_combination[1]

    chl1_path = os.path.join('./tmp/', chl1)
    chl2_path = os.path.join('./tmp/', chl2)
    with open(chl1_path, 'r') as f1, open(chl2_path, 'r') as f2:
        
        # Extract coordinates of atoms A, B, C, D from data1
        data1 = np.loadtxt(f1)
        A1, B1, C1, D1 = data1[0],data1[1],data1[2],data1[3]
        # Center of vCA in Chlorophyll 1
        r1 = (A1 + C1) / 2

        # Vectors for Chlorophyll 1 
        vCA1 = (A1 - C1) / np.linalg.norm(A1 - C1)
        vDB1 = (B1 - D1) / np.linalg.norm(B1 - D1)
        N1 = vnorm1 = np.cross(vCA1, vDB1) / np.linalg.norm(np.cross(vCA1, vDB1))

        # Axis for Chlorophyll 1
        z_axis1 = vnorm1
        y_axis1 = vCA1
        x_axis1 = np.cross(vCA1, vnorm1) / np.linalg.norm(np.cross(vCA1, vnorm1))

        # Chlorophyll 2 
        data2 = np.loadtxt(f2)
        A2, B2, C2, D2 = data2[0],data2[1],data2[2],data2[3]

        # Center of vCA in Chlorophyll 2
        r2 = (A2 + C2) / 2

        # Vectors for Chlorophyll 2 
        vCA2 = (A2 - C2) / np.linalg.norm(A2 - C2)
        vDB2 = (B2 - D2) / np.linalg.norm(B2 - D2)
        N2 = vnorm2 = np.cross(vCA2, vDB2) / np.linalg.norm(np.cross(vCA2, vDB2))

        # Axis for Chlorophyll 2
        z_axis2 = vnorm2
        y_axis2 = vCA2
        x_axis2 = np.cross(vCA2, vnorm2) / np.linalg.norm(np.cross(vCA2, vnorm2))

        #translation values 
        dxdydz = r1 - r2
        dx = dxdydz[0]
        dy = dxdydz[1]
        dz = dxdydz[2]
        q = np.linalg.norm(dxdydz)
        translation = [dz, dy, dx]
        # First chlorophyll centered at the same location as the second chlorophyll
        data1 = data1 - (dxdydz)
     
        # Move both chlorophylls to origin
        mol1 = np.array(data1 - r2)
        mol2 = np.array(data2 - r2)

        R1 = rotate_to_global(mol1)
        R2 = rotate_to_global(mol2)

        R_relative = np.dot(R1.T, R2)

        euler = rotation_matrix_to_euler_angles(R_relative)
        table = [['CHL1', 'CHL2', 'dZ', 'dY', 'dX', 'Distance Between Molecules', 'Z(psi)', 'Y(theta)', 'X(phi)'], [chl1, chl2, dx, dy, dz, q, np.degrees(euler[0]), np.degrees(euler[1]),np.degrees(euler[2])]]
        print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

╒═══════════╤═══════════╤══════╤══════╤══════╤══════════════════════════════╤══════════╤════════════╤══════════╕
│ CHL1      │ CHL2      │   dZ │   dY │   dX │   Distance Between Molecules │   Z(psi) │   Y(theta) │   X(phi) │
╞═══════════╪═══════════╪══════╪══════╪══════╪══════════════════════════════╪══════════╪════════════╪══════════╡
│ CHL_1.txt │ CHL_2.txt │    0 │    0 │    0 │                            0 │ -10.0015 │         -0 │        0 │
╘═══════════╧═══════════╧══════╧══════╧══════╧══════════════════════════════╧══════════╧════════════╧══════════╛
